In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [2]:
ex1 = pd.read_parquet('train/1.회원정보/2018_회원정보_전체.parquet')

In [3]:
all_cols = ex1.columns.tolist()
# print("전체 컬럼 이름:")
# print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
# print("\n문자형 컬럼 이름:")
# print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
# print("\n숫자형 컬럼 이름:")
# print(num_cols)

In [4]:
cols = ['가입통신회사코드', '거주시도명', '직장시도명', '_1순위신용체크구분', '_2순위신용체크구분', '연회비발생카드수_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M', 'Life_Stage']  # 실제 열 이름으로 바꿔주세요

In [5]:
mapping = {
    'L사': 0,
    'K사': 1,
    'S사': 2
}
ex1['가입통신회사코드'] = (
    ex1['가입통신회사코드']
      .map(mapping)        # L사→0, K사→1, S사→2, 그 외→NaN
      .fillna(-1)          # NaN→-1
      .astype(int)         # 정수형
)

In [6]:
ex1['연령'] = ex1['연령'].astype(str).str.replace('대', '', regex=False)

# 2) 숫자로 변환, 에러는 NaN→fillna(-1)→정수형
ex1['연령'] = (
    pd.to_numeric(ex1['연령'], errors='coerce')
      .fillna(-1)
      .astype(int)
)


In [7]:

regions = [
    '서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종',
    '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'
]

region_type = CategoricalDtype(categories=regions, ordered=True)

for col in ['거주시도명', '직장시도명']:
    ex1[col] = (
        ex1[col]
           .astype(str)                   # 혹시 숫자 타입 섞여 있어도 문자열로
           .astype(region_type)           # 카테고리로 변환
           .cat.codes                     # 0~16, 리스트 외 값은 -1
    )

In [8]:
count_cols = [
    '연회비발생카드수_B0M',
    '상품관련면제카드수_B0M',
    '임직원면제카드수_B0M',
    '우수회원면제카드수_B0M',
    '기타면제카드수_B0M'
]

for col in count_cols:
    ex1[col] = (
        ex1[col]
          .astype(str)  # None → 'None', NaN → 'nan'
          .str.replace('개', '', regex=False)  # '0개' → '0', 'None' → 'None'
    )
    # 숫자로 변환, 에러는 NaN 처리 → NaN → -1 로 채우기 → 정수형
    ex1[col] = (
        pd.to_numeric(ex1[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [9]:
le_cols = ['_1순위신용체크구분', '_2순위신용체크구분', 'Life_Stage']

for col in le_cols:
    codes, uniques = pd.factorize(ex1[col], sort=True)
    ex1[col] = codes 

In [10]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '남녀구분코드', '연령', 'Segment', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '가입통신회사코드', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '거주시도명', '직장시도명', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용카드수_체크_가족', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '이용금액_R3M_체크_가족', '_1순위카드이용금액', '_1순위카드이용건수', '_1순위신용체크구분', '_2순위카드이용금액', '_2순위카드이용건수', '_2순위신용체크구분', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '연회비발생카드수_B0M', '연회비할인카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '청구금액_기본연회

In [11]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()
low_var = num_df.var()[ num_df.var() < 0.001 ].index.tolist()
low_var

['회원여부_이용가능',
 '소지여부_신용',
 '유효카드수_체크_가족',
 '이용가능카드수_체크_가족',
 '이용카드수_체크_가족',
 '이용금액_R3M_체크_가족',
 '연회비할인카드수_B0M',
 '할인금액_기본연회비_B0M',
 '할인금액_제휴연회비_B0M',
 '상품관련면제카드수_B0M',
 '임직원면제카드수_B0M',
 '우수회원면제카드수_B0M',
 '기타면제카드수_B0M']

In [12]:
missing_mask = ex1.isna() | (ex1 == -1)

missing_ratio = missing_mask.mean()

high_na = missing_ratio[missing_ratio > 0.3].index.tolist()

print("결측(또는 -1) 비율 > 30% 컬럼:", high_na)

결측(또는 -1) 비율 > 30% 컬럼: ['_2순위신용체크구분']


In [13]:
cols_to_drop = [
    '회원여부_이용가능',
    '소지여부_신용',
    '유효카드수_체크_가족',
    '이용가능카드수_체크_가족',
    '이용카드수_체크_가족',
    '이용금액_R3M_체크_가족',
    '연회비할인카드수_B0M',
    '할인금액_기본연회비_B0M',
    '할인금액_제휴연회비_B0M',
    '상품관련면제카드수_B0M',
    '임직원면제카드수_B0M',
    '우수회원면제카드수_B0M',
    '기타면제카드수_B0M',
    '_2순위신용체크구분'
]
ex1.drop(columns=cols_to_drop, inplace=True)

In [14]:
ex1

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지카드수_유효_신용,소지카드수_이용가능_신용,입회일자_신용,...,이용가능여부_해외겸용_신용_본인,이용여부_3M_해외겸용_신용_본인,연회비발생카드수_B0M,기본연회비_B0M,제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,TRAIN_000000,2,40,D,1,0,1,1,20130101,...,0,0,0,0,0,0,0,0,5,22
1,201807,TRAIN_000001,1,30,E,1,1,1,1,20170801,...,0,0,0,0,0,0,0,0,4,18
2,201807,TRAIN_000002,1,30,C,1,0,1,1,20080401,...,0,0,0,0,0,0,0,0,6,20
3,201807,TRAIN_000003,2,40,D,1,0,2,2,20160501,...,1,1,0,0,0,0,0,1,5,17
4,201807,TRAIN_000004,2,40,E,1,1,1,1,20180601,...,0,0,0,0,0,0,0,1,4,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,2,-1,E,1,1,1,1,20010701,...,1,0,0,0,0,0,0,0,1,39
2399996,201812,TRAIN_399996,2,50,D,1,1,1,1,20170701,...,1,1,0,0,0,0,0,0,5,24
2399997,201812,TRAIN_399997,1,30,C,1,0,1,1,20090501,...,1,1,0,0,0,0,0,0,6,18
2399998,201812,TRAIN_399998,1,40,E,1,1,1,1,20130101,...,0,0,0,0,0,0,0,0,4,27


In [15]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)


C:\Users\AHN\AppData\Local\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\AHN\AppData\Local\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


VIF > 10인 컬럼들: ['소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '수신거부여부_DM', '수신거부여부_메일', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '_1순위카드이용금액', '_2순위카드이용금액', '_2순위카드이용건수', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '기본연회비_B0M', '제휴연회비_B0M', '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '최종카드발급경과월']
               feature           VIF
4          소지카드수_유효_신용  1.679884e+01
5        소지카드수_이용가능_신용  1.457110e+01
6              입회일자_신용  5.690636e+02
7           입회경과개월수_신용  5.700802e+02
12           수신거부여부_DM  1.533111e+01
13           수신거부여부_메일  1.246080e+01
23          유효카드수_신용체크           inf
24            유효카드수_신용           inf
25         유효카드수_신용_가족  1.335482e+02
26            유효카드수_체

In [17]:
cols_to_drop = ['유효카드수_신용','유효카드수_체크','이용가능카드수_신용','이용가능카드수_체크','이용카드수_체크','이용카드수_신용','청구금액_제휴연회비_B0M','청구금액_기본연회비_B0M']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

C:\Users\AHN\AppData\Local\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF > 10인 컬럼들: ['소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '수신거부여부_DM', '수신거부여부_메일', '유효카드수_신용체크', '유효카드수_신용_가족', '이용가능카드수_신용체크', '이용가능카드수_신용_가족', '이용카드수_신용_가족', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '_1순위카드이용금액', '_2순위카드이용금액', '_2순위카드이용건수', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '최종카드발급경과월']
               feature           VIF
4          소지카드수_유효_신용  1.541969e+01
5        소지카드수_이용가능_신용  1.423517e+01
6              입회일자_신용  5.690297e+02
7           입회경과개월수_신용  5.700551e+02
12           수신거부여부_DM  1.532845e+01
13           수신거부여부_메일  1.246055e+01
23          유효카드수_신용체크  3.580399e+01
24         유효카드수_신용_가족  1.335453e+02
25        이용가능카드수_신용체크  3.595767e+01
26       이용가능카드수_신용_가족  1.422155e+02
28         이용카드수_신용_가족  2.142891e+01
29       이용금액_R3M_신용체크  1.021186e+10
30         이용금액_R3M_신용  9.156338e+09
31      이용금액_R3M_신용_가족  1.536062e+01
32     

In [18]:
cols_to_drop = ['이용금액_R3M_신용체크']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

C:\Users\AHN\AppData\Local\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF > 10인 컬럼들: ['소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '수신거부여부_DM', '수신거부여부_메일', '유효카드수_신용체크', '유효카드수_신용_가족', '이용가능카드수_신용체크', '이용가능카드수_신용_가족', '이용카드수_신용_가족', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '_1순위카드이용금액', '_2순위카드이용금액', '_2순위카드이용건수', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '최종카드발급경과월']
               feature         VIF
4          소지카드수_유효_신용   15.373002
5        소지카드수_이용가능_신용   14.211925
6              입회일자_신용  569.021166
7           입회경과개월수_신용  570.046837
12           수신거부여부_DM   15.328359
13           수신거부여부_메일   12.460507
23          유효카드수_신용체크   35.802510
24         유효카드수_신용_가족  133.544854
25        이용가능카드수_신용체크   35.953866
26       이용가능카드수_신용_가족  142.215452
28         이용카드수_신용_가족   21.427781
29         이용금액_R3M_신용   70.659901
30      이용금액_R3M_신용_가족   15.360550
32          _1순위카드이용금액   40.347061
35          _2순위카드이용금액   27.308322
36          _2순위카드이용건수   15.136939

In [19]:
cols_to_drop = ['입회경과개월수_신용']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

C:\Users\AHN\AppData\Local\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF > 10인 컬럼들: ['소지카드수_유효_신용', '소지카드수_이용가능_신용', '수신거부여부_DM', '수신거부여부_메일', '유효카드수_신용체크', '유효카드수_신용_가족', '이용가능카드수_신용체크', '이용가능카드수_신용_가족', '이용카드수_신용_가족', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '_1순위카드이용금액', '_2순위카드이용금액', '_2순위카드이용건수', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '최종카드발급경과월']
               feature         VIF
4          소지카드수_유효_신용   15.372209
5        소지카드수_이용가능_신용   14.211736
11           수신거부여부_DM   15.328352
12           수신거부여부_메일   12.460151
22          유효카드수_신용체크   35.802188
23         유효카드수_신용_가족  133.544850
24        이용가능카드수_신용체크   35.953695
25       이용가능카드수_신용_가족  142.215022
27         이용카드수_신용_가족   21.427240
28         이용금액_R3M_신용   70.659068
29      이용금액_R3M_신용_가족   15.358925
31          _1순위카드이용금액   40.345957
34          _2순위카드이용금액   27.306189
35          _2순위카드이용건수   15.130757
36      최종유효년월_신용_이용가능   12.325750
37        최종유효년월_신용_이용   11.124955
38            최종카드발급일자  

In [20]:
cols_to_drop = ['이용가능여부_해외겸용_신용_본인']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

C:\Users\AHN\AppData\Local\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF > 10인 컬럼들: ['소지카드수_유효_신용', '소지카드수_이용가능_신용', '수신거부여부_DM', '수신거부여부_메일', '유효카드수_신용체크', '유효카드수_신용_가족', '이용가능카드수_신용체크', '이용가능카드수_신용_가족', '이용카드수_신용_가족', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '_1순위카드이용금액', '_2순위카드이용금액', '_2순위카드이용건수', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '최종카드발급경과월']
               feature         VIF
4          소지카드수_유효_신용   15.367683
5        소지카드수_이용가능_신용   14.204138
11           수신거부여부_DM   15.328351
12           수신거부여부_메일   12.460143
22          유효카드수_신용체크   35.801811
23         유효카드수_신용_가족  133.544092
24        이용가능카드수_신용체크   35.953679
25       이용가능카드수_신용_가족  142.214031
27         이용카드수_신용_가족   21.427137
28         이용금액_R3M_신용   70.657545
29      이용금액_R3M_신용_가족   15.358923
31          _1순위카드이용금액   40.344533
34          _2순위카드이용금액   27.306181
35          _2순위카드이용건수   15.130747
36      최종유효년월_신용_이용가능   12.325701
37        최종유효년월_신용_이용   11.124862
38            최종카드발급일자   17.189009
39        

In [21]:
cols_to_drop = ['이용가능카드수_신용_가족','이용가능여부_해외겸용_본인','유효카드수_신용_가족']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

C:\Users\AHN\AppData\Local\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF > 10인 컬럼들: ['소지카드수_유효_신용', '소지카드수_이용가능_신용', '수신거부여부_DM', '수신거부여부_메일', '유효카드수_신용체크', '이용가능카드수_신용체크', '이용카드수_신용_가족', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '_1순위카드이용금액', '_2순위카드이용금액', '_2순위카드이용건수', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '최종카드발급경과월']
               feature        VIF
4          소지카드수_유효_신용  15.243605
5        소지카드수_이용가능_신용  14.133857
11           수신거부여부_DM  15.327773
12           수신거부여부_메일  12.459846
22          유효카드수_신용체크  35.797557
23        이용가능카드수_신용체크  35.945726
25         이용카드수_신용_가족  14.146298
26         이용금액_R3M_신용  70.582377
27      이용금액_R3M_신용_가족  14.159389
29          _1순위카드이용금액  40.255348
32          _2순위카드이용금액  27.298412
33          _2순위카드이용건수  15.105126
34      최종유효년월_신용_이용가능  12.322676
35        최종유효년월_신용_이용  11.124621
36            최종카드발급일자  17.187653
39     보유여부_해외겸용_신용_본인  22.739142
40  이용여부_3M_해외겸용_신용_본인  27.373642
46           최종카드발급경과월  19.822862


In [23]:
cols_to_drop = ['소지카드수_유효_신용','수신거부여부_DM','유효카드수_신용체크','이용금액_R3M_신용','_2순위카드이용금액','최종유효년월_신용_이용가능','이용여부_3M_해외겸용_신용_본인','최종카드발급경과월']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

C:\Users\AHN\AppData\Local\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF > 10인 컬럼들: ['이용카드수_신용_가족', '이용금액_R3M_신용_가족']
           feature        VIF
22     이용카드수_신용_가족  13.923428
23  이용금액_R3M_신용_가족  14.134762


In [24]:
cols_to_drop = ['이용금액_R3M_신용_가족']
ex1.drop(columns=cols_to_drop, inplace=True)

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)

C:\Users\AHN\AppData\Local\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF > 10인 컬럼들: []
Empty DataFrame
Columns: [feature, VIF]
Index: []


In [25]:
ex1.to_parquet('회원정보_전처리.parquet', index=False)